In [1]:
import requests

import urllib.request
import json 

In [9]:
# Пример данных
data = (13,7,2012,39,45,13,0.6,85.2,11.3,10.4,4.2,10.9,4.7,1.0)
print(data)

# формируем запрос
def send_json(x):
    day,month,year,Temperature,RH,Ws,Rain,FMC,DMC,DC,ISI,BUI,FWI,region_A = x
    body = {'day': day, 
            'month': month, 
            'year': year, 
            'Temperature': Temperature, 
            'RH': RH, 
            'Ws': Ws, 
            'Rain': Rain, 
            'FMC': FMC,
            'DMC': DMC, 
            'DC': DC, 
            'ISI': ISI, 
            'BUI': BUI, 
            'FWI': FWI,
            'region_A': region_A+.1e-4}
    myurl = 'http://127.0.0.1:5000/'+'predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

# обращение к серверу с запросом из одного набора (его построили руками выше - data)
if __name__ == '__main__':
    response = send_json(data)
    print('предсказание',response)

(13, 7, 2012, 39, 45, 13, 0.6, 85.2, 11.3, 10.4, 4.2, 10.9, 4.7, 1.0)
предсказание 0.948182613756884


Сделаем обработку массы запросов:

1. загрузим данные X_test

In [10]:
import pandas as pd
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [11]:
response = send_json(X_test[['day','month','year','Temperature','RH','Ws','Rain','FMC'\
            ,'DMC','DC','ISI','BUI','FWI','region_A']].iloc[0,:])
response

0.0002953354859920759

3. Сделаем N запросов и оценим время

In [12]:
X_test.head()

,day,month,year,Temperature,RH,Ws,Rain,FMC,DMC,DC,ISI,BUI,FWI,region_A
0,25,7,2012,39,64,9,1.2,73.8,11.7,15.9,1.1,11.4,0.7,1.0
1,20,7,2012,36,50,16,0.0,89.9,32.7,71.0,9.5,32.6,17.3,1.0
2,15,9,2012,24,82,15,0.4,44.9,0.9,7.3,0.2,1.4,0.0,0.0
3,30,8,2012,35,70,17,0.8,72.7,25.2,180.4,1.7,37.4,4.2,0.0
4,3,9,2012,25,78,15,3.8,42.6,1.2,7.5,0.1,1.7,0.0,0.0


In [13]:
N = 50

In [14]:
send_json(X_test[['day','month','year','Temperature','RH','Ws','Rain','FMC'\
            ,'DMC','DC','ISI','BUI','FWI','region_A']]\
            .iloc[:1,:].iloc[0,:])

0.0002953354859920759

In [15]:
%%time

predictions = X_test[['day','month','year','Temperature','RH','Ws','Rain','FMC'\
            ,'DMC','DC','ISI','BUI','FWI','region_A']]\
            .iloc[:N,:].apply(lambda x: send_json(x),axis=1)

CPU times: user 109 ms, sys: 15.1 ms, total: 124 ms
Wall time: 1.31 s


In [16]:
predictions.values[:5]

array([2.95335486e-04, 9.99999997e-01, 1.19015536e-08, 1.52664050e-01,
       2.68839899e-07])

In [17]:
y_test.head()

,Classes
0,not fire
1,fire
2,not fire
3,not fire
4,not fire


In [18]:
from sklearn.metrics import roc_auc_score,roc_curve

In [19]:
y_test = pd.DataFrame(y_test).applymap(lambda x: x.replace(" ", ""))
y_test = y_test.applymap(lambda x: 1 if x=='fire' else 0)

In [20]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

1.0

так как данные очень простые, метрика показывает отличное качество модели